<a href="https://colab.research.google.com/github/kalebsampaco/Tensor-flow-2-completo/blob/master/Collab_13_Preparar_un_modelo_de_TensorFlow_para_m%C3%B3viles_con_TensorFlow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno de trabajo

In [0]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 49kB/s 
     |████████████████████████████████| 3.2MB 18.2MB/s 
     |████████████████████████████████| 368kB 32.8MB/s 
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


## Paso 2: Importar las dependencias

In [0]:
import tensorflow as tf
import numpy as np

#from tensorflow.keras.datasets import fashion_mnist
from keras.datasets import fashion_mnist
tf.__version__

'1.13.1'

## Paso 3: Pre procesado de datos

### Cargar el dataset de FashionMNIST

In [0]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 1s 0us/step


### Normalizar las imágenes

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

### Redimensionar las imágenes

In [0]:
X_train.shape

(60000, 28, 28)

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [0]:
X_train.shape

(60000, 784)

## Paso 4: Construir el modelo

### Defining the model

In [0]:
model = tf.keras.models.Sequential()

### Definir el modelo

In [0]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compilar el modelo

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Entrenar el modelo

In [0]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 9s 156us/sample - loss: 0.5310 - sparse_categorical_accuracy: 0.8114
Epoch 2/5
60000/60000 [==============================] - 8s 134us/sample - loss: 0.3987 - sparse_categorical_accuracy: 0.8560
Epoch 3/5
60000/60000 [==============================] - 8s 137us/sample - loss: 0.3664 - sparse_categorical_accuracy: 0.8666
Epoch 4/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.3448 - sparse_categorical_accuracy: 0.8733
Epoch 5/5
60000/60000 [==============================] - 8s 139us/sample - loss: 0.3293 - sparse_categorical_accuracy: 0.8780


### Evaluar el modelo

In [0]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Precisión del conjunto de test: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 73us/sample - loss: 0.3602 - sparse_categorical_accuracy: 0.8726
Precisión del conjunto de test: 0.8726000189781189


## Paso 5: Convertir el modelo a la versión de TensorFlow Lite

### Guardar el modelo

In [0]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Crear el TFLite Converter

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


### Convertir el modelo

In [0]:
tflite_model = converter.convert()

### Guardar la versión de TFLite version del modelo

In [0]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)